In [1]:
import chzphot_v02 as cp
%pylab
from scipy import *
#from numpy import *
import scipy.ndimage as ndi

from scipy.signal import medfilt
from scipy.optimize import leastsq

# import pyfits as pf
import astropy.io.fits as pf

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
RawStarMap = pf.getdata('150501.007.fits')

In [4]:
figure(1)
imshow(RawStarMap)
colorbar()

<matplotlib.colorbar.Colorbar instance at 0x107b84290>

In [5]:
#Do below with localized maps? split into subimages first?
StarMapFilt = medfilt(RawStarMap, 3)
m1 = median(StarMapFilt)  #filtered median
dif1 = abs(StarMapFilt - m1)
MAD1 = median(dif1)  #MAD of filtered
sd1 = 1.4826 * MAD1  #standard dev from MAD

thresh = 5.0
GoodPix1 = where(abs(StarMapFilt - m1) < (thresh*sd1))
m2 = mean(StarMapFilt[GoodPix1])
sd2 = std(StarMapFilt[GoodPix1])

GoodPix2 = where(abs(StarMapFilt - m2) < (thresh*sd2))
m3 = mean(StarMapFilt[GoodPix2])
sd3 = std(StarMapFilt[GoodPix2])

print(m3)  #mean noise

RedNoiseMap = 1.0 * StarMapFilt  # copy filtered star map
RedNoiseMap[GoodPix2] = m3  # replace background with mean noise

400.216871065


In [6]:
matshow(RedNoiseMap)

In [7]:
#Pick a few stars to centroid
Star1 = RedNoiseMap[20:40,260:280] * 1.0
Star2 = RedNoiseMap[320:335,855:870] * 1.0
Star3 = RedNoiseMap[795:810,755:775] * 1.0
Star4 = RedNoiseMap[475:495,600:615] * 1.0

In [8]:
#Guesses for y,x center, subR window radius
yg1, xg1, subR1 = (30., 270., 15.)
yg2, xg2, subR2 = (327.,857., 15.)
yg3, xg3, subR3 = (803.,765., 15.)
yg4, xg4, subR4 = (485.,608, 15.)

In [9]:
#Reduce background to zero
NoNoiseMap = RedNoiseMap - m3

In [10]:
#Run Centroiding Function:
x1c, y1c = cp.gcntrd(NoNoiseMap, subR1, xg1, yg1)
x2c, y2c = cp.gcntrd(NoNoiseMap, subR2, xg2, yg2)
x3c, y3c = cp.gcntrd(NoNoiseMap, subR3, xg3, yg3)
x4c, y4c = cp.gcntrd(NoNoiseMap, subR4, xg4, yg4)

print(x1c, y1c)
print(x2c, y2c)
print(x3c, y3c)
print(x4c, y4c)

(269.96924532244481, 28.841605862191717)
(864.31580358739075, 328.52538114998856)
(763.95418008472768, 801.80713476252402)
(606.23227574332236, 484.93417352236105)


In [11]:
#show graph of each star w/ Gaussian center marked

Star1c = NoNoiseMap[20:40,260:280] * 1.0
Star2c = NoNoiseMap[320:335,855:870] * 1.0
Star3c = NoNoiseMap[795:810,755:775] * 1.0
Star4c = NoNoiseMap[475:495,600:615] * 1.0

close('all')

matshow(Star1c)
x1cscaled = x1c - 260.
y1cscaled = y1c - 20.
scatter([x1cscaled], [y1cscaled])
imshow(Star1c)

matshow(Star2c)
x2cscaled = x2c - 855.
y2cscaled = y2c - 320.
scatter([x2cscaled], [y2cscaled])
imshow(Star2c)

matshow(Star3c)
x3cscaled = x3c - 755.
y3cscaled = y3c - 795.
scatter([x3cscaled], [y3cscaled])
imshow(Star3c)

matshow(Star4c)
x4cscaled = x4c - 600.
y4cscaled = y4c - 475.
scatter([x4cscaled], [y4cscaled])
imshow(Star4c)